In [ ]:
import pandas as pd
df = pd.read_excel("YouTube-videos-new.xlsx")

# 应用该函数到 DataFrame 的 URL 列
df['video_id'] = df['视频网址'].apply(lambda x : x[x.find('v=')+2:])
df.head(3)

# df.to_csv('YouTube-videos-new.csv', index=False,encoding='utf-8-sig')
# df.to_excel('YouTube-videos-new.xlsx', index=False)

d:\Python\envs\scrap\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


# 读取json

In [2]:
# import json

# # 读取JSON文件
# with open("data\comment_100\*.json", "r") as file:
#     data = json.load(file)

# # 提取"items"字段对应的值
# items = data["items"]

# # 将数据转换为DataFrame
# df = pd.DataFrame(items)

In [3]:
import json
import glob
import os

# 定义匹配JSON文件的模式
pattern = os.path.join("data", "comment_100", "*.json")

# 获取所有匹配的JSON文件列表
json_files = glob.glob(pattern)

# 初始化一个空的DataFrame
df = pd.DataFrame()

# 遍历所有JSON文件
for json_file in json_files:
    # 读取每个JSON文件
    with open(json_file, "r", encoding='utf-8') as file:
        data = json.load(file)
        
        # 提取"items"字段对应的值
        items = data["items"]
        
        # 将数据转换为临时DataFrame
        temp_df = pd.DataFrame(items)
        
        # 合并到主DataFrame中
        df = pd.concat([df, temp_df], ignore_index=True)



In [4]:
# df.sample(5)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3471 entries, 0 to 3470
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   kind     3471 non-null   object
 1   etag     3471 non-null   object
 2   id       3471 non-null   object
 3   snippet  3471 non-null   object
 4   replies  1397 non-null   object
dtypes: object(5)
memory usage: 135.7+ KB


用户名，发表评论的时间，点赞数量，具体评论

In [5]:
# find wanted fields in `snippet`
df.iloc[0,3]

{'channelId': 'UC1soWWg79S9TUdSjzAHHftw',
 'videoId': '-XcGW_L1N0k',
 'topLevelComment': {'kind': 'youtube#comment',
  'etag': 'dZfmAuWSKwl3JfHDEc5ippdRkbI',
  'id': 'Ugx365--jk4WSqtsdO14AaABAg',
  'snippet': {'channelId': 'UC1soWWg79S9TUdSjzAHHftw',
   'videoId': '-XcGW_L1N0k',
   'textDisplay': 'Thanks for watching<a href="UC1soWWg79S9TUdSjzAHHftw/JkxFY6LmDY2A8gPWhLZI"></a> <br>Subscribe to our channel for more videos in China🇨🇳 Our China playlist: <a href="https://www.youtube.com/playlist?list=PL9QdAxhqglB93tWtt3PsWWSOJycnPNj0m">https://youtube.com/playlist?list=PL9QdAxhqglB93tWtt3PsWWSOJycnPNj0m&amp;si=ydl-nM1ITJOaOnT6</a>',
   'textOriginal': 'Thanks for watching \nSubscribe to our channel for more videos in China🇨🇳 Our China playlist: https://youtube.com/playlist?list=PL9QdAxhqglB93tWtt3PsWWSOJycnPNj0m&si=ydl-nM1ITJOaOnT6',
   'authorDisplayName': '@JetLagWarriors',
   'authorProfileImageUrl': 'https://yt3.ggpht.com/vmfmpvBgVWylDuTXSR4p4t76Ax0GFlBikWngY4dAoxj_911Jm2hOfy-cZ7egdRar

# 提取顶级评论

In [6]:
def get_basic(x):
    y = pd.Series()
    y["video_id"] = x["snippet"].get("topLevelComment").get("snippet").get("videoId")
    y["content"] = x["snippet"].get("topLevelComment").get("snippet").get("textDisplay")
    y["review_id"] =  x["snippet"].get("topLevelComment").get("id")# 应为对与二级评论，它带有parent_id,我们可以通过这个parent_id找到对应的review_id
    y["parent_id"] = 0  # 应为是1级评论，所以没有parent_id 都赋值为0
    y["reviewer_name"] = x["snippet"].get("topLevelComment").get("snippet").get("authorDisplayName")
    y["published_time"] = x["snippet"].get("topLevelComment").get("snippet").get("publishedAt")
    y["updated_time"] = x["snippet"].get("topLevelComment").get("snippet").get("updatedAt")
    y["total_reply_count"] = x["snippet"].get("totalReplyCount")  # 用户针对顶级评论提交的回复总数。
    y["level"] = 1    # 顶级评论（1级评论）
    y["likecount"]  = x["snippet"].get("topLevelComment").get("snippet").get("likeCount")
    return y

df_basic = df.apply(get_basic,axis = 1)

In [7]:
df_basic.head(3)
df_basic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3471 entries, 0 to 3470
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   video_id           3471 non-null   object
 1   content            3471 non-null   object
 2   review_id          3471 non-null   object
 3   parent_id          3471 non-null   int64 
 4   reviewer_name      3471 non-null   object
 5   published_time     3471 non-null   object
 6   updated_time       3471 non-null   object
 7   total_reply_count  3471 non-null   int64 
 8   level              3471 non-null   int64 
 9   likecount          3471 non-null   int64 
dtypes: int64(4), object(6)
memory usage: 271.3+ KB


# 提取对顶级评论的回复

level：一级评论为0，二级评论为1（并且在`parent_id`标注其父评论）

find wanted fields in `replies`: replies 对象是一个容器，其中包含评论的回复列表（如果存在）。但是该列表中包含的回复数量有限。除非列表中的项数等于 snippet.totalReplyCount 属性的值，否则该回复列表只是顶级评论可用回复总数的一部分。要检索顶级评论的所有回复，您需要调用 comments.list 方法并使用 parentId 请求参数来标识要检索其回复的评论


In [8]:
tmp = df.iloc[0,4]

tmp.get("comments")[0]

{'kind': 'youtube#comment',
 'etag': '2W65SK9Rq3J-LCgj9Zka2MxYC_s',
 'id': 'Ugx365--jk4WSqtsdO14AaABAg.9yEBPJSu2169yELPQoNoH_',
 'snippet': {'channelId': 'UC1soWWg79S9TUdSjzAHHftw',
  'videoId': '-XcGW_L1N0k',
  'textDisplay': 'Your video is going to be banded.  Americans is not suppose to know how great other countries are!',
  'textOriginal': 'Your video is going to be banded.  Americans is not suppose to know how great other countries are!',
  'parentId': 'Ugx365--jk4WSqtsdO14AaABAg',
  'authorDisplayName': '@Asian_Connection',
  'authorProfileImageUrl': 'https://yt3.ggpht.com/EFQEWV5G5EEQU2uuUntVArtHThFIMRDFL8HrPdV1jYm_aDS9vssGt4_cFDV_qRK2fq_HCly8=s48-c-k-c0x00ffffff-no-rj',
  'authorChannelUrl': 'http://www.youtube.com/@Asian_Connection',
  'authorChannelId': {'value': 'UC2oV2ooEoEwSanCB8jSvGnA'},
  'canRate': True,
  'viewerRating': 'none',
  'likeCount': 49,
  'publishedAt': '2023-12-12T18:29:20Z',
  'updatedAt': '2023-12-12T18:29:20Z'}}

In [9]:
def get_reply(df_raw):
    df_output = pd.DataFrame()
    replies_list = df.loc[~df["replies"].isnull(), "replies"]
    if len(replies_list) == 0:
        print("No replies in this df_raw.")
        return
    for x in replies_list:
        x = x.get("comments")
        for reply in x:
            y = pd.Series()
            y["video_id"] = reply.get("snippet").get("videoId")
            y["content"] = reply.get("snippet").get("textDisplay")
            y["review_id"] =  reply.get("id")
            y["parent_id"] = reply.get("snippet").get("parentId")  # 应为是2级评论，所以有parent_id
            y["reviewer_name"] = reply.get("snippet").get("authorDisplayName")
            y["published_time"] = reply.get("snippet").get("publishedAt")
            y["updated_time"] = reply.get("snippet").get("updatedAt")
            y["total_reply_count"] = 0  # 应为是二级评论，没有对他们回复，所以total_reply_count为0
            y["level"] = 2    # 应为都是对顶级评论的回复，所以都是二级（后续在其中鉴别三级的）
            y["likecount"]  = reply.get("snippet").get("likeCount")
            df_output = df_output._append(y,ignore_index=True)
    return df_output


df_replies = get_reply(df)
df_replies.head(3)

,video_id,content,review_id,parent_id,reviewer_name,published_time,updated_time,total_reply_count,level,likecount
0,-XcGW_L1N0k,Your video is going to be banded. Americans i...,Ugx365--jk4WSqtsdO14AaABAg.9yEBPJSu2169yELPQoNoH_,Ugx365--jk4WSqtsdO14AaABAg,@Asian_Connection,2023-12-12T18:29:20Z,2023-12-12T18:29:20Z,0,2,49
1,-XcGW_L1N0k,"By reading through the policy details, looks l...",Ugx365--jk4WSqtsdO14AaABAg.9yEBPJSu2169yEUOfTALe-,Ugx365--jk4WSqtsdO14AaABAg,@joey3291,2023-12-12T19:47:53Z,2023-12-12T19:47:53Z,0,2,11
2,-XcGW_L1N0k,answer from AI: Under the 144-hour visa-free t...,Ugx365--jk4WSqtsdO14AaABAg.9yEBPJSu2169yEVr8OO8wH,Ugx365--jk4WSqtsdO14AaABAg,@joey3291,2023-12-12T20:00:39Z,2023-12-12T20:00:39Z,0,2,6


In [10]:
df_final = pd.concat([df_basic,df_replies])
df_final.head()

,video_id,content,review_id,parent_id,reviewer_name,published_time,updated_time,total_reply_count,level,likecount
0,-XcGW_L1N0k,"Thanks for watching<a href=""UC1soWWg79S9TUdSjz...",Ugx365--jk4WSqtsdO14AaABAg,0,@JetLagWarriors,2023-12-12T17:01:57Z,2023-12-15T16:58:14Z,69,1,476
1,-XcGW_L1N0k,China has alot to offer as a travel destinatio...,Ugzlz1-2S8BB6B-onVh4AaABAg,0,@dunkierookie970,2023-12-12T13:50:11Z,2023-12-12T16:58:25Z,569,1,3449
2,-XcGW_L1N0k,"In Tokyo, there are also many cameras on the s...",UgyelK4yZ9kg_K17jSZ4AaABAg,0,@shinha,2023-12-12T15:41:11Z,2023-12-12T15:41:11Z,370,1,2627
3,-XcGW_L1N0k,I go to China every year. Chinese cities are v...,UgziXalu_BzOKodbxXZ4AaABAg,0,@yvesklein8197,2024-03-07T06:58:07Z,2024-03-07T06:58:07Z,28,1,174
4,-XcGW_L1N0k,"我相信任何人来了中国,感受这片土地感受的文化底蕴都会被所吸引，充分感受了各地不同的风景后无法...",Ugz82CgNmEQXqP5aZ8d4AaABAg,0,@muye4244,2024-02-27T23:49:55Z,2024-02-27T23:49:55Z,0,1,4


In [15]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7836 entries, 0 to 4364
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   video_id           7836 non-null   object
 1   content            7836 non-null   object
 2   review_id          7836 non-null   object
 3   parent_id          7836 non-null   object
 4   reviewer_name      7836 non-null   object
 5   published_time     7836 non-null   object
 6   updated_time       7836 non-null   object
 7   total_reply_count  7836 non-null   int64 
 8   level              7836 non-null   int64 
 9   likecount          7836 non-null   int64 
dtypes: int64(3), object(7)
memory usage: 673.4+ KB


In [11]:
# df_final.to_excel("comment_final.xlsx", index=False)

# 识别回复中可能的二级评论

In [12]:
# df_replies.loc[lambda x :x["content"].str.contains("@"),:]["content"].to_dict()